In [6]:
import sys
import numpy as np 
sys.path.append('../../../src/')
sys.path.append('../../../src/wikidata')

from mask_dataset import MaskedDataset
dataset = MaskedDataset(model_name="mbert", reload=False)

Load pre-saved file /home/xzhao/workspace/probing-mulitlingual/result/dataset-mbert/mlama-dataset.pkl for MaskedDataset instance


In [50]:
from pred_evaluation import get_all_and_matched_uuid_lsts
from wiki_evaluation import _get_subject_object_cooccurence_in_article
import random 

uuid_infos = dataset.get_uuid_info_all_lang()
def find_optimal_uuid(uuids, lang, size=1):
    cand_uuids = []
    for uuid in uuids:
        uuid_info = uuid_infos[uuid][lang]
        sub_token = uuid_info['sub']
        obj_token = uuid_info['obj']        
        if obj_token.lower() not in sub_token.lower():
            cand_uuids.append(uuid)

    
    if cand_uuids != []:
        cand_uuids = uuids
    
    if len(cand_uuids) <= size:
        return cand_uuids
    else:
        return random.choices(cand_uuids, k=size)

def collect_absent_predictable_facts(dataset, display_content=False):
    uuid2wikimatches = _get_subject_object_cooccurence_in_article(dataset, dataset.langs)
    rel2predicted_uuid = get_all_and_matched_uuid_lsts(dataset=dataset, reload=False)[1]    
    lang2uuids = {}
    for lang in dataset.langs:
        uuid2present = uuid2wikimatches[lang]
        rel2predictable = rel2predicted_uuid[lang]

        predicted_uuids = []
        for rel in rel2predictable:
            predicted_uuids.extend(rel2predictable[rel])
        
        nonwiki_uuids = [uuid for uuid in predicted_uuids if uuid2present[uuid] == 0]
        for rel in rel2predictable:
            if len(rel2predictable[rel]) > 0:
                nonwiki_uuids_by_rel = [uuid for uuid in rel2predictable[rel] if uuid in nonwiki_uuids]
                if nonwiki_uuids_by_rel:
                    uuid = find_optimal_uuid(nonwiki_uuids_by_rel, lang)
                    lang2uuids.setdefault(lang, []).extend(uuid)
    return lang2uuids
lang2absent_uuids = collect_absent_predictable_facts(dataset)                    

In [65]:
import pandas as pd
df = pd.DataFrame(columns=["Language", "Absent & Predictable fact", "Corresponding English"])

for lang in lang2absent_uuids:
    uuids = find_optimal_uuid(lang2absent_uuids[lang], lang, size=3)
    for uuid in uuids:
        item = {
            "Language": dataset.display_lang(lang, prefix=False),
            "Absent & Predictable fact": dataset.display_uuid(uuid, lang),
            "Corresponding English": dataset.display_uuid(uuid, 'en')
        }
        df = pd.concat([df, pd.DataFrame(item, index=[0])])

In [66]:
df

,Language,Absent & Predictable fact,Corresponding English
0,Malay,Jablanica District terletak di Serbia.,Jablanica District is located in Serbia .
0,Malay,Bahasa rasmi Payerne ialah Bahasa Perancis.,The official language of Payerne is French .
0,Malay,Optik kuantum adalah sebahagian daripada fizik.,quantum optics is part of physics .
0,Malay,Paul Meurice dulu bekerja di Paris.,Paul Meurice used to work in Paris .
0,Malay,Raja Farouk I berafiliasi dengan agama Islam.,Farouk I of Egypt is affiliated with the Islam...
...,...,...,...
0,Romanian,Noua filozofie franceză a fost creat în Franța.,New Philosophers was created in France .
0,Romanian,Claude Fleury a murit în Paris.,Claude Fleury died in Paris .
0,Romanian,Etilbenzen este format din carbon.,ethylbenzene consists of carbon .
0,Romanian,Democrație iliberală este o subclasă a lui dem...,illiberal democracy is a subclass of democracy .


In [68]:
df.to_csv("./absent_predictable_matches.csv", index=False)